In [1]:
from collections import namedtuple

# Define Point class
class Point:
    def __init__(self, x, y):
        self.x = x
        self.y = y

    def __repr__(self):
        return f"Point({self.x}, {self.y})"

# Define MoveTo class
class MoveTo:
    def __init__(self, *args):
        if isinstance(args[0], Point):
            self.endPoint = args[0]
        elif isinstance(args[0], list):
            self.endPoint = Point(args[0][0], args[0][1])
        elif isinstance(args, tuple):
            self.endPoint = Point(args[0], args[1])

    def __repr__(self):
        return f"MoveTo({self.endPoint})"

# Define LineTo class
class LineTo:
    def __init__(self, *args):
        if isinstance(args[0], Point):
            self.endPoint = args[0]
        elif isinstance(args[0], list):
            self.endPoint = Point(args[0][0], args[0][1])
        elif isinstance(args, tuple):
            self.endPoint = Point(args[0], args[1])

    def __repr__(self):
        return f"LineTo({self.endPoint})"

# Define CurveTo class
class CurveTo:
    def __init__(self, *args):
        if isinstance(args[0], Point):
            self.controlPoint = args[0]
            self.endPoint = args[1]
        elif isinstance(args[0], list) and isinstance(args[1], list):
            self.controlPoint = Point(args[0][0], args[0][1])
            self.endPoint = Point(args[1][0], args[1][1])
        elif isinstance(args, tuple):
            self.controlPoint = Point(args[0], args[1])
            self.endPoint = Point(args[2], args[3])

    def __repr__(self):
        return f"CurveTo({self.controlPoint}, {self.endPoint})"

# Example usage
move_to = MoveTo(1, 2)
line_to = LineTo([3, 4])
curve_to = CurveTo([5, 6], [7, 8])

print(move_to)
print(line_to)
print(curve_to)

MoveTo(Point(1, 2))
LineTo(Point(3, 4))
CurveTo(Point(5, 6), Point(7, 8))


In [5]:
# Define Edge class
import re
class Edge:
    def __init__(self, commands, leftFill=None, rightFill=None):
        self.commands = commands
        self.leftFill = leftFill
        self.rightFill = rightFill

    @classmethod
    def fromXFL(cls, edgeElem, fills):
        edge = cls(cls.edgeData2arr(edgeElem['edges']))
        if 'fillStyle0' in edgeElem:
            edge.leftFill = fills[int(edgeElem['fillStyle0']) - 1]
        if 'fillStyle1' in edgeElem:
            edge.rightFill = fills[int(edgeElem['fillStyle1']) - 1]
        edge.simplify()
        return edge

    @property
    def startPoint(self):
        return self.commands[0].endPoint

    @property
    def endPoint(self):
        return self.commands[-1].endPoint

    @staticmethod
    def hexToNum(hexstr):
        num = int(hexstr.replace('.', ''), 16)
        if num >= int('80000000', 16):
            num = -((1 << 32) - num)
        return num / 256.0

    @staticmethod
    def edgeData2arr(edgeData):
        commands = []
        for opcode, params in re.findall(r'([!|/\[])([^!|/\[]+)', edgeData):
            numbers = [Edge.hexToNum(num) if prefix == '#' else float(num) / 20 for prefix, num in re.findall(r'(#?)(\S+)\s*', params)]
            if opcode == '!':
                commands.append(MoveTo(*numbers))
            elif opcode == '|':
                commands.append(LineTo(*numbers))
            elif opcode == '[':
                commands.append(CurveTo(*numbers))
        return commands

    def simplify(self):
        self.commands = [cmd for i, cmd in enumerate(self.commands) if not (isinstance(cmd, MoveTo) and i > 0 and self.commands[i-1].endPoint == cmd.endPoint)]
        return self

    def reverse(self):
        revEdge = [MoveTo(self.endPoint)]
        revCmds = self.commands[::-1]
        for i, cmd in enumerate(revCmds[:-1]):
            newCmd = cmd.__class__(*cmd.__dict__.values())
            newCmd.endPoint = revCmds[i+1].endPoint
            revEdge.append(newCmd)
        return Edge(revEdge, self.rightFill, self.leftFill)

    def append(self, otherEdge):
        result = Edge(self.commands + otherEdge.commands[1:], self.leftFill, self.rightFill)
        return result

    def closed(self):
        return self.endPoint == self.startPoint

# Example usage
edgeElem = {
    'edges': '! 0 0 | 100 100 [ 50 50 100 100',
    'fillStyle0': '1',
    'fillStyle1': '2'
}
fills = ['fill1', 'fill2']
edge = Edge.fromXFL(edgeElem, fills)
print(edge)
print(edge.startPoint)
print(edge.endPoint)
print(edge.closed())

Point(0.0, 0.0)
Point(5.0, 5.0)
False


In [7]:
import xml.etree.ElementTree as ET

# Assuming previously defined Point, MoveTo, LineTo, CurveTo, and Edge classes

class Symbol:
    def __init__(self, name, edges, fillStyles):
        self.name = name
        self.edges = edges
        self.fillStyles = fillStyles

    @classmethod
    def fromXFL(cls, doc):
        fills = cls.readFillStyles(doc)
        edges = [Edge.fromXFL(edge, fills) for edge in doc.findall('.//xfl:Edge[@edges]', namespaces={'xfl': 'http://ns.adobe.com/xfl/2008/'})]
        return cls(doc.getroot().attrib['name'], edges, fills)

    @staticmethod
    def readFillStyles(doc):
        fills = []
        for elem in doc.findall('.//xfl:fills/xfl:FillStyle', namespaces={'xfl': 'http://ns.adobe.com/xfl/2008/'}):
            index = int(elem.attrib['index']) - 1
            color = elem.find('xfl:SolidColor', namespaces={'xfl': 'http://ns.adobe.com/xfl/2008/'})
            if color is not None:
                fills.insert(index, {'color': color.attrib['color']})
        return fills

    def addEdgeToGroup(self, edgeGroups, group, edge):
        if group not in edgeGroups:
            edgeGroups[group] = []
        edgeGroups[group].append(edge)

    def findConnectedTo(self, prevEdge, edges):
        for index, edge in enumerate(edges):
            if edge.startPoint == prevEdge.endPoint:
                return index
        return -1

    def findContour(self, group):
        contour = group.pop(0)
        while not contour.closed() and len(group) > 0:
            at = self.findConnectedTo(contour, group)
            if at == -1:
                break
            contour = contour.append(group[at])
            group.pop(at)
        return contour

    def findContours(self, group):
        contours = []
        while len(group) > 0:
            contours.append(self.findContour(group))
        return contours

    def filledRegions(self):
        edgeGroups = {}
        for edge in self.edges:
            if edge.leftFill is not None:
                self.addEdgeToGroup(edgeGroups, edge.leftFill, edge)
            if edge.rightFill is not None:
                self.addEdgeToGroup(edgeGroups, edge.rightFill, edge.reverse())

        filledRegs = {}
        for fillStyle, group in edgeGroups.items():
            filledRegs[fillStyle] = self.findContours(group)
        return filledRegs

# Example usage
xml_content = '''<root name="example">
    <fills>
        <FillStyle index="1">
            <SolidColor color="#FF0000"/>
        </FillStyle>
        <FillStyle index="2">
            <SolidColor color="#00FF00"/>
        </FillStyle>
    </fills>
    <Edge edges="! 0 0 | 100 100 [ 50 50 100 100" fillStyle0="1" fillStyle1="2"/>
</root>'''

doc = ET.ElementTree(ET.fromstring(xml_content))
symbol = Symbol.fromXFL(doc)
print(symbol.name)
print(symbol.filledRegions())

example
{}
